# Creating database
We will create a database with exported csv file from previous script analysis.

## Create connection to database
Using sqlalchemy to stablish connection to database.

In [12]:
import sqlalchemy

# This part of code is private
username = "-----"
password = "-----"
server = "-----"
database = "-----"

engine_stmt = "mysql+mysqldb://{}:{}@{}:3306/{}".format(username,password,server,database)
engine = sqlalchemy.create_engine(engine_stmt)

## Date variable to export db
We save date to use in prices tables. 

In [15]:
import datetime
date = str(datetime.datetime.now())[:10]
date

'2022-04-19'

## Load gas_prices.csv as dataframe
We load the first csv file with pandas dataframe.

In [22]:
import pandas as pd
df = pd.read_csv("export_files\gas_prices_20220415_160024.csv")
df.set_index("place_id", inplace=True)
df

,name,cree_id,latitude,longitude,regular,premium,diesel
place_id,,,,,,,
2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.92140,22.49,24.99,21.99
2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.30370,-99.74484,21.25,23.19,23.09
2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.45140,17.28,19.20,NaN
2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.30420,21.79,24.39,23.54
2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977,17.79,19.99,NaN
...,...,...,...,...,...,...,...
27758,SUMINISTRO DE COMBUSTIBLES EL SEIS S. DE R.L. ...,PL/23961/EXP/ES/2022,20.88375,-102.70250,NaN,NaN,NaN
27759,GASOLINERA SACSALUM SA DE CV,PL/23960/EXP/ES/2022,16.32370,-91.97680,NaN,NaN,NaN
27764,"AEROPUERTO INTERNACIONAL FELIPE ANGELES, SOCIE...",PL/23974/EXP/ES/2022,19.72579,-99.03263,NaN,NaN,NaN


### Creating gas stations dataframe
Our first table consist on gas stations places information.

In [23]:
df_places = df.iloc[:,:4]
df_places

,name,cree_id,latitude,longitude
place_id,,,,
2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,32.47641,-116.92140
2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,20.30370,-99.74484
2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,31.71947,-106.45140
2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,20.71413,-103.30420
2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,26.03787,-98.29977
...,...,...,...,...
27758,SUMINISTRO DE COMBUSTIBLES EL SEIS S. DE R.L. ...,PL/23961/EXP/ES/2022,20.88375,-102.70250
27759,GASOLINERA SACSALUM SA DE CV,PL/23960/EXP/ES/2022,16.32370,-91.97680
27764,"AEROPUERTO INTERNACIONAL FELIPE ANGELES, SOCIE...",PL/23974/EXP/ES/2022,19.72579,-99.03263


#### Upload gas stations dataframe to db

In [24]:
df_places.to_sql(name="gas_stations",
                 con=engine,
                 if_exists="replace")

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\engine\default.py", line 682, in do_rollback
    dbapi_connection.rollback()
MySQLdb._exceptions.OperationalError: (2006, 'Server has gone away')


12959

#### Set primary key

In [25]:
with engine.connect() as con:
    con.execute(
        """
        ALTER TABLE gas_stations
        ADD PRIMARY KEY (place_id);
        """)

#### Drop gas_stations table

In [18]:
with engine.connect() as con:
    con.execute(
        """
        DROP TABLE gas_stations;
        """
    )

### Creating regular dataframe
We create three tables for prices (regular, premium and diesel).

In [26]:
df_regular = df.loc[:,["regular"]]
df_regular.rename(columns={"regular":date},inplace=True)
df_regular

,2022-04-19
place_id,
2039,22.49
2040,21.25
2041,17.28
2042,21.79
2043,17.79
...,...
27758,NaN
27759,NaN
27764,NaN


#### Upload regular dataframe to db

In [27]:
df_regular.to_sql(name="regular",
                 con=engine,
                 if_exists="replace")

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\engine\default.py", line 682, in do_rollback
    dbapi_connection.rollback()
MySQLdb._exceptions.OperationalError: (2006, 'Server has gone away')


12959

#### Set primary key

In [28]:
with engine.connect() as con:
    con.execute(
        """
        ALTER TABLE regular
        ADD PRIMARY KEY (place_id);
        """)

#### Drop regular table

In [17]:
with engine.connect() as con:
    con.execute(
        """
        DROP TABLE regular;
        """
    )

### Creating premium dataframe

In [29]:
df_premium = df.loc[:, ["premium"]]
df_premium.rename(columns={"premium": date}, inplace=True)
df_premium

,2022-04-19
place_id,
2039,24.99
2040,23.19
2041,19.20
2042,24.39
2043,19.99
...,...
27758,NaN
27759,NaN
27764,NaN


#### Upload premium dataframe to db

In [30]:
df_premium.to_sql(name="premium",
                 con=engine,
                 if_exists="replace")

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\engine\default.py", line 682, in do_rollback
    dbapi_connection.rollback()
MySQLdb._exceptions.OperationalError: (2006, 'Server has gone away')


12959

#### Set primary key

In [31]:
with engine.connect() as con:
    con.execute(
        """
        ALTER TABLE premium
        ADD PRIMARY KEY (place_id);
        """)

#### Drop premium table

In [ ]:
with engine.connect() as con:
    con.execute(
        """
        DROP TABLE premium;
        """
    )

### Creating diesel dataframe

In [32]:
df_diesel = df.loc[:, ["diesel"]]
df_diesel.rename(columns={"diesel": date}, inplace=True)
df_diesel

,2022-04-19
place_id,
2039,21.99
2040,23.09
2041,NaN
2042,23.54
2043,NaN
...,...
27758,NaN
27759,NaN
27764,NaN


#### Upload premium dataframe to db

In [33]:
df_diesel.to_sql(name="diesel",
                 con=engine,
                 if_exists="replace")

12959

#### Set primary key

In [ ]:
with engine.connect() as con:
    con.execute(
        """
        ALTER TABLE diesel
        ADD PRIMARY KEY (place_id);
        """)

#### Drop diesel table

In [ ]:
with engine.connect() as con:
    con.execute(
        """
        DROP TABLE diesel;
        """
    )

## Load gas_address.csv
Second csv file that we will upload to database and contains address for gas stations.

In [19]:
df = pd.read_csv("export_files\gas_address_20220415_160024.csv")
df.set_index("place_id",inplace=True)
df

,amenity,road,hamlet,city,county,state,ISO3166-2-lvl4,postcode
place_id,,,,,,,,
2039,Pemex,Boulevard de los Insurgentes,El Porvenir,Tijuana,Municipio de Tijuana,Baja California,MX-BCN,22223
2040,NaN,MEX 45,NaN,NaN,Nopala de Villagrán,Hidalgo,MX-HID,NaN
2041,Pemex E02526,Avenida Licenciado Adolfo López Mateos,NaN,Ciudad Juárez,Municipio de Juárez,Chihuahua,MX-CHH,94158
2042,Pemex,Anillo Periférico Norte Manuel Gómez Morín,NaN,Guadalajara,NaN,Jalisco,MX-JAL,44390
2043,NaN,NaN,NaN,Reynosa,Municipio de Reynosa,Tamaulipas,MX-TAM,88500
...,...,...,...,...,...,...,...,...
27758,NaN,Carretera Tepatitlán-Las Adjuntas,NaN,NaN,San Ignacio Cerro Gordo,Jalisco,MX-JAL,47601
27759,NaN,Chiapas 218,NaN,NaN,Las Margaritas,Chiapas,MX-CHP,30187
27764,NaN,Acceso a Aeropuerto Int. Felipe Angeles,NaN,NaN,Nextlalpan,Estado de México,MX-MEX,55789


#### Upload station address dataframe to db

In [20]:
df.to_sql(name="station_address",
          con=engine,
          if_exists="replace")

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\pool\base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\chccr\AppData\Local\Programs\Python\Python39\lib\site-packages\sqlalchemy\engine\default.py", line 682, in do_rollback
    dbapi_connection.rollback()
MySQLdb._exceptions.OperationalError: (2006, 'Server has gone away')


12848

#### Set primary key

In [21]:
with engine.connect() as con:
    con.execute(
        """
        ALTER TABLE station_address
        ADD PRIMARY KEY (place_id);
        """)

#### Drop station address table

In [ ]:
with engine.connect() as con:
    con.execute(
        """
        DROP TABLE station_address;
        """
    )